In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import rasterio as rio

from pathlib import Path
from src.functions import *
import matplotlib.pyplot as plt
import cubo
import xarray as xr
import rioxarray as rxr
from pyproj import Transformer

This example shows how to process Sentinel 2 index mosaics available [here](https://ckan.ymparisto.fi/dataset/sentinel-2-image-index-mosaics-s2ind-sentinel-2-kuvamosaiikit-s2ind) into gapless mosaics and derive yearly statistics using Paituli STAC.

First, set the url for Paituli stac.

In [2]:
paituli_stac_url = 'https://paituli.csc.fi/geoserver/ogc/stac/v1'


Use [`cubo`](https://cubo.readthedocs.io/en/latest/) to easily create the required data cube. Cubo requires the coordinates to be in `EPSG:4326` so we need a transformer.

In [3]:
t = Transformer.from_crs('EPSG:3067', 'EPSG:4326')

lat, lon = t.transform(457325, 7702545)

In [4]:
da = cubo.create(
    lat=lat, # Central latitude of the cube
    lon=lon, # Central longitude of the cube
    collection="sentinel_2_monthly_index_mosaics_at_fmi", # Name of the STAC collection
    bands=["ndvi"], # Bands to retrieve
    start_date="2016-04-01", # Start date of the cube
    end_date="2025-05-31", # End date of the cube
    edge_size=4000, # Edge size of the cube (px)
    resolution=10, # Pixel size of the cube (m),
    stac=paituli_stac_url
)

Now we have a datacube. As it is a dask array, the data is processed only after the graphs are created.

In [5]:
da

<xarray.DataArray 'sentinel_2_monthly_index_mosaics_at_fmi' (time: 117,
                                                             band: 1, y: 4000,
                                                             x: 4000)> Size: 15GB
dask.array<fetch_raster_window, shape=(117, 1, 4000, 4000), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/13)
  * time                       (time) datetime64[ns] 936B 2016-04-01 ... 2024...
    id                         (time) <U49 23kB 'Sentinel-2_indeksimosaiikki_...
  * band                       (band) <U4 16B 'ndvi'
  * x                          (x) float64 32kB 4.373e+05 ... 4.773e+05
  * y                          (y) float64 32kB 7.723e+06 ... 7.683e+06
    start_datetime             (time) <U29 14kB '2016-04-01T00:00:00.000+00:0...
    ...                         ...
    proj:code                  <U9 36B 'EPSG:3067'
    end_datetime               (time) <U29 14kB '2016-04-30T23:59:59.000+00:0...
    proj:transform             object 8B {0, 1, 7800000, 10, 50000, -10}
    title                      <U4 16B 'ndvi'
    epsg                       int64 8B 32635
    cubo:distance_from_center  (y, x) float64 128MB 2.828e+04 ... 2.827e+04
Attributes:
    collection:           sentinel_2_monthly_index_mosaics_at_fmi
    stac:                 https://paituli.csc.fi/geoserver/ogc/stac/v1
    epsg:                 32635
    resolution:           10
    edge_size:            4000
    central_lat:          69.42870189667391
    central_lon:          25.911697105643754
    central_y:            7702545.000115293
    central_x:            457325.0000006164
    time_coverage_start:  2016-04-01
    time_coverage_end:    2025-05-31

## Create base mosaics for spring and autumn

Seasonal base mosaics are used to fill gaps for April, May and October, as they are usually the most cloudy months. Base mosaics are constructed from the median values of April, May and October during the full observation period.

First filter `DataArrays` based the timesteps for spring and autumn. Spring is April and May, autumn is mid-September to October.

In [6]:
time = da.indexes['time'].values 

year = da.time.dt.year
month = da.time.dt.month
day = da.time.dt.day

spring_mask = (month == 4) | ((month == 5) & (day == 1))
autumn_mask = (month == 10) | ((month == 9) & ((day == 30) | (day == 15)))

spring = da.sel(time=spring_mask)
autumn = da.sel(time=autumn_mask)

In [7]:
spring

<xarray.DataArray 'sentinel_2_monthly_index_mosaics_at_fmi' (time: 27, band: 1,
                                                             y: 4000, x: 4000)> Size: 3GB
dask.array<getitem, shape=(27, 1, 4000, 4000), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/13)
  * time                       (time) datetime64[ns] 216B 2016-04-01 ... 2024...
    id                         (time) <U49 5kB 'Sentinel-2_indeksimosaiikki_2...
  * band                       (band) <U4 16B 'ndvi'
  * x                          (x) float64 32kB 4.373e+05 ... 4.773e+05
  * y                          (y) float64 32kB 7.723e+06 ... 7.683e+06
    start_datetime             (time) <U29 3kB '2016-04-01T00:00:00.000+00:00...
    ...                         ...
    proj:code                  <U9 36B 'EPSG:3067'
    end_datetime               (time) <U29 3kB '2016-04-30T23:59:59.000+00:00...
    proj:transform             object 8B {0, 1, 7800000, 10, 50000, -10}
    title                      <U4 16B 'ndvi'
    epsg                       int64 8B 32635
    cubo:distance_from_center  (y, x) float64 128MB 2.828e+04 ... 2.827e+04
Attributes:
    collection:           sentinel_2_monthly_index_mosaics_at_fmi
    stac:                 https://paituli.csc.fi/geoserver/ogc/stac/v1
    epsg:                 32635
    resolution:           10
    edge_size:            4000
    central_lat:          69.42870189667391
    central_lon:          25.911697105643754
    central_y:            7702545.000115293
    central_x:            457325.0000006164
    time_coverage_start:  2016-04-01
    time_coverage_end:    2025-05-31

In [8]:
autumn

<xarray.DataArray 'sentinel_2_monthly_index_mosaics_at_fmi' (time: 18, band: 1,
                                                             y: 4000, x: 4000)> Size: 2GB
dask.array<getitem, shape=(18, 1, 4000, 4000), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/13)
  * time                       (time) datetime64[ns] 144B 2016-09-15 ... 2024...
    id                         (time) <U49 4kB 'Sentinel-2_indeksimosaiikki_2...
  * band                       (band) <U4 16B 'ndvi'
  * x                          (x) float64 32kB 4.373e+05 ... 4.773e+05
  * y                          (y) float64 32kB 7.723e+06 ... 7.683e+06
    start_datetime             (time) <U29 2kB '2016-09-15T00:00:00.000+00:00...
    ...                         ...
    proj:code                  <U9 36B 'EPSG:3067'
    end_datetime               (time) <U29 2kB '2016-10-15T23:59:59.000+00:00...
    proj:transform             object 8B {0, 1, 7800000, 10, 50000, -10}
    title                      <U4 16B 'ndvi'
    epsg                       int64 8B 32635
    cubo:distance_from_center  (y, x) float64 128MB 2.828e+04 ... 2.827e+04
Attributes:
    collection:           sentinel_2_monthly_index_mosaics_at_fmi
    stac:                 https://paituli.csc.fi/geoserver/ogc/stac/v1
    epsg:                 32635
    resolution:           10
    edge_size:            4000
    central_lat:          69.42870189667391
    central_lon:          25.911697105643754
    central_y:            7702545.000115293
    central_x:            457325.0000006164
    time_coverage_start:  2016-04-01
    time_coverage_end:    2025-05-31

Base mosaics are median values for these timesteps.

In [9]:
base_spring = spring.sel(band='ndvi').median(dim='time', skipna=True)
base_autumn = autumn.sel(band='ndvi').median(dim='time', skipna=True)

In [10]:
base_spring

<xarray.DataArray 'sentinel_2_monthly_index_mosaics_at_fmi' (y: 4000, x: 4000)> Size: 128MB
dask.array<nanmedian, shape=(4000, 4000), dtype=float64, chunksize=(788, 788), chunktype=numpy.ndarray>
Coordinates:
    band                       <U4 16B 'ndvi'
  * x                          (x) float64 32kB 4.373e+05 ... 4.773e+05
  * y                          (y) float64 32kB 7.723e+06 ... 7.683e+06
    gsd                        float64 8B 10.0
    proj:code                  <U9 36B 'EPSG:3067'
    proj:transform             object 8B {0, 1, 7800000, 10, 50000, -10}
    title                      <U4 16B 'ndvi'
    epsg                       int64 8B 32635
    cubo:distance_from_center  (y, x) float64 128MB 2.828e+04 ... 2.827e+04

In [11]:
base_spring.rio.write_crs('EPSG:3067', inplace=True)
#base_spring.astype(np.uint8).rio.to_raster('stac_data/spring_base.tif') # Uncomment to save the file

<xarray.DataArray 'sentinel_2_monthly_index_mosaics_at_fmi' (y: 4000, x: 4000)> Size: 128MB
dask.array<nanmedian, shape=(4000, 4000), dtype=float64, chunksize=(788, 788), chunktype=numpy.ndarray>
Coordinates:
    band                       <U4 16B 'ndvi'
  * x                          (x) float64 32kB 4.373e+05 ... 4.773e+05
  * y                          (y) float64 32kB 7.723e+06 ... 7.683e+06
    gsd                        float64 8B 10.0
    proj:code                  <U9 36B 'EPSG:3067'
    proj:transform             object 8B {0, 1, 7800000, 10, 50000, -10}
    title                      <U4 16B 'ndvi'
    epsg                       int64 8B 32635
    cubo:distance_from_center  (y, x) float64 128MB 2.828e+04 ... 2.827e+04
    spatial_ref                int64 8B 0

In [12]:
base_autumn.rio.write_crs('EPSG:3067', inplace=True)
#base_autumn.astype(np.uint8).rio.to_raster('stac_data/autumn_base.tif')  # Uncomment to save the file

<xarray.DataArray 'sentinel_2_monthly_index_mosaics_at_fmi' (y: 4000, x: 4000)> Size: 128MB
dask.array<nanmedian, shape=(4000, 4000), dtype=float64, chunksize=(965, 965), chunktype=numpy.ndarray>
Coordinates:
    band                       <U4 16B 'ndvi'
  * x                          (x) float64 32kB 4.373e+05 ... 4.773e+05
  * y                          (y) float64 32kB 7.723e+06 ... 7.683e+06
    gsd                        float64 8B 10.0
    proj:code                  <U9 36B 'EPSG:3067'
    proj:transform             object 8B {0, 1, 7800000, 10, 50000, -10}
    title                      <U4 16B 'ndvi'
    epsg                       int64 8B 32635
    cubo:distance_from_center  (y, x) float64 128MB 2.828e+04 ... 2.827e+04
    spatial_ref                int64 8B 0

## Fill gaps  with the maximum value of previous two years

Use the data from full months, i.e. between the first and the last day.

As this step needs two previous years of data, the earliest year that can be processed from Sentinel-2 data is 2018.

In [13]:
da = da.sel(time=da.time.dt.day==1)

Reshape to `year`, `month` indexable DataArray.

In [14]:
da = da.assign_coords(year=da.time.dt.year, month=da.time.dt.month)
da = da.set_index(time=['year', 'month']).unstack('time')

Create a `DataArray` for filled data.

In [15]:
da_filled = da.copy()

In [16]:
da_y1 = da.shift(year=1)
da_y2 = da.shift(year=2)
prev_max = xr.concat([da_y1, da_y2], dim='source').max(dim='source', skipna=True)
da_filled = xr.where(da_filled.notnull(), da_filled, prev_max)

## Fill April and May with spring mosaic

In [17]:
spring_msk = da_filled.month.isin([4,5])

da_filled = xr.where(da_filled.notnull() | spring_msk, da_filled, base_spring)

## Fill October with autumn mosaic

In [18]:
aut_msk = da_filled.month == 10
da_filled = xr.where(da_filled.notnull() | aut_msk, da_filled, base_autumn)

## Fill growth season months with mean values of previous and next month of the same year

There are some areas where growth season has gaps, and they are filled with the mean values of the two adjacent months.

In [19]:
da_m1 = da_filled.shift(month=1)
da_m2 = da_filled.shift(month=-1)
monthmeans = xr.concat([da_m1, da_m2], dim='source').mean(dim='source', skipna=True)

gr_mask = da_filled.month.isin([6,7,8])

da_filled = xr.where(da_filled.notnull() | gr_mask, da_filled, monthmeans)

In [20]:
da_filled = da_filled.sel(year=da_filled.year >= 2018)
da_filled.rio.write_crs('EPSG:3067', inplace=True)

<xarray.DataArray (band: 1, y: 4000, x: 4000, year: 7, month: 7)> Size: 6GB
dask.array<getitem, shape=(1, 4000, 4000, 7, 7), dtype=float64, chunksize=(1, 788, 788, 1, 1), chunktype=numpy.ndarray>
Coordinates: (12/15)
  * year                       (year) int64 56B 2018 2019 2020 ... 2023 2024
  * month                      (month) int64 56B 4 5 6 7 8 9 10
    id                         (year, month) <U49 10kB 'Sentinel-2_indeksimos...
  * band                       (band) <U4 16B 'ndvi'
  * x                          (x) float64 32kB 4.373e+05 ... 4.773e+05
  * y                          (y) float64 32kB 7.723e+06 ... 7.683e+06
    ...                         ...
    end_datetime               (year, month) <U29 6kB '2018-04-30T23:59:59.00...
    proj:transform             object 8B {0, 1, 7800000, 10, 50000, -10}
    title                      <U4 16B 'ndvi'
    epsg                       int64 8B 32635
    cubo:distance_from_center  (y, x) float64 128MB 2.828e+04 ... 2.827e+04
    spatial_ref                int64 8B 0

If the intermediate data need to be saved, it is done by uncommenting the following lines.

In [21]:
#da_filled = da_filled.compute()
#da_filled = da_filled.stack(time=['year', 'month']).sortby('time')
#da_filled.rio.write_crs('EPSG:3067', inplace=True)
#for t in da_filled['time'].values:
#    da_t = da_filled.sel(time=t, band='ndvi')
#    fname = f"stac_data/filled/filled_{t[0]}_{t[1]}.tif" 
#    da_t.rio.to_raster(fname)

## Derive the statistics

Max, mean, median and sum are really straightforward.

In [22]:
da_filled = da_filled.chunk({'x':1024, 'y':1024, 'month':-1})

yearly_max = da_filled.max(dim='month', skipna=True)
yearly_mean = da_filled.mean(dim='month', skipna=True)
yearly_median = da_filled.median(dim='month', skipna=True)
yearly_sum =  da_filled.sum(dim='month', skipna=True)

In [23]:
for y in yearly_max.year.values:
    yearly_max.sel(year=y, band='ndvi').compute().rio.write_crs('EPSG:3067').astype(np.uint8).rio.to_raster(f'stac_data/stats/max_{y}.tif')

In [24]:
for y in yearly_mean.year.values:
    yearly_mean.sel(year=y, band='ndvi').rio.write_crs('EPSG:3067').astype(np.uint8).rio.to_raster(f'stac_data/stats/mean_{y}.tif')

In [25]:
for y in yearly_median.year.values:
    yearly_median.sel(year=y, band='ndvi').rio.write_crs('EPSG:3067').astype(np.uint8).rio.to_raster(f'stac_data/stats/median_{y}.tif')

In [26]:
for y in yearly_sum.year.values:
    yearly_sum.sel(year=y, band='ndvi').rio.write_crs('EPSG:3067').astype(np.uint16).rio.to_raster(f'stac_data/stats/sum_{y}.tif')

Quantiles, on the other hand, not so much. `fastnanquantile` makes computing them a lot faster.

In [27]:
from fastnanquantile import xrcompat

First, the DataArray needs to be rechunked.

In [28]:
da_q = da_filled.chunk({'month':-1})


Then `xrcompat.xr_apply_nanquantile` can be used efficiently. Derive Q10 and Q25.

In [29]:
yearly_q10 = xrcompat.xr_apply_nanquantile(da_q, q=0.1, dim='month')


In [30]:
for y in yearly_q10.year.values:
    yearly_q10.sel(year=y, band='ndvi').rio.write_crs('EPSG:3067').astype(np.uint8).rio.to_raster(f'stac_data/stats/q10_{y}.tif')

In [31]:
yearly_q25 = xrcompat.xr_apply_nanquantile(da_q, q=0.25, dim='month')
for y in yearly_q25.year.values:
    yearly_q25.sel(year=y, band='ndvi').rio.write_crs('EPSG:3067').astype(np.uint8).rio.to_raster(f'stac_data/stats/q25_{y}.tif')

Amplitude is $max - yearly\_q25$

In [32]:
yearly_amplitude = yearly_max - yearly_q25

In [33]:
for y in yearly_amplitude.year.values:
    yearly_amplitude.sel(year=y, band='ndvi').rio.write_crs('EPSG:3067').astype(np.int16).rio.to_raster(f'stac_data/stats/amp_{y}.tif')